In [25]:
import numpy as np
import os
import simtk.unit as unit
from simtk import openmm
import logging
from perses.app.setup_relative_calculation import run_setup
from tqdm import tqdm
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [3]:
# Read in htf
with open("/data/chodera/zhangi/perses_benchmark/fah/13413/RUNS/RUN3911/htf.npz", 'rb') as f:
    htf = np.load(f, allow_pickle=True)
    htf = htf.get('arr_0')

In [12]:
htf = htf.flatten()[0]

In [5]:
temperature = 300
nequil = 100
n_steps_per_iteration = 250
platform_name = 'OpenCL'
timestep = 2.*unit.femtosecond
collision_rate = 90./unit.picosecond

In [20]:
from openmmtools.integrators import LangevinIntegrator
_logger.info(f'Starting to relax')
integrator = LangevinIntegrator(temperature=temperature, timestep=timestep, collision_rate=collision_rate)
platform = openmm.Platform.getPlatformByName(platform_name)


2020-07-13 16:29:37,509:(13.32s):root:Starting to relax


In [21]:
# prepare the plaform
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(htf.hybrid_system, integrator, platform)
context.setPeriodicBoxVectors(*htf.hybrid_system.getDefaultPeriodicBoxVectors())
context.setPositions(htf.hybrid_positions)


In [22]:
_logger.info(f'Starting to minimise')
openmm.LocalEnergyMinimizer.minimize(context)


2020-07-13 16:29:53,772:(16.26s):root:Starting to minimise


In [27]:
# Equilibrate
_logger.info(f'set velocities to temperature')
context.setVelocitiesToTemperature(temperature)
_logger.info(f'Starting to equilibrate for {nequil*n_steps_per_iteration*timestep}')
for i in tqdm(range(nequil)):
    integrator.step(n_steps_per_iteration)
context.setVelocitiesToTemperature(temperature)
state = context.getState(getEnergy=True, getForces=True, getPositions=True, getVelocities=True, getParameters=True)
_logger.info(f'Relax done')


2020-07-13 16:31:11,185:(4.29s):root:set velocities to temperature
2020-07-13 16:31:11,208:(0.02s):root:Starting to equilibrate for 50000.0 fs
100%|██████████| 100/100 [02:31<00:00,  1.51s/it]
2020-07-13 16:33:42,389:(151.18s):root:Relax done


In [28]:
from perses.utils import data
data.serialize(state, f"state.xml.bz2")

In [31]:
with open('state.xml', 'w') as outfile:
    state_xml = XmlSerializer.serialize(state)
    outfile.write(state_xml)

In [ ]:
import bz2
with bz2.open('state.xml.bz2', 'rb') as outfile:
    serialized_thing = XmlSerializer.serialize(item)
    outfile.write(serialized_thing.encode())

In [30]:
from simtk.openmm import XmlSerializer
# Load state and set box vectors, positions, and velocities
with open("state.xml.bz2", 'rb') as infile:
    state = XmlSerializer.deserialize(infile.read().decode())

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc6 in position 10: invalid continuation byte

In [32]:
from simtk.openmm import XmlSerializer
# Load state and set box vectors, positions, and velocities
with open("state.xml", 'r') as infile:
    state = XmlSerializer.deserialize(infile.read())

In [33]:
state

<simtk.openmm.openmm.State; proxy of <Swig Object of type 'OpenMM::State *' at 0x2adc7a588480> >

In [ ]:
## NOTE the problem was i wasnt using bz2.open() for state.xml